### Some small additions to the dataframe
For ease of plotting in 2b

In [1]:
from functions import map_scatter, glasbey
from matplotlib.ticker import LogLocator, ScalarFormatter, NullFormatter
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
from google.cloud import storage
from scipy import stats
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
from anndata import AnnData
import pandas as pd
import numpy as np
import seaborn as sns
import colorcet as cc
import scanpy as sc
import copy
import os

In [2]:
# read in dimred_clstr data
data_path = "gs://mayo-nguyen-transplant/nduarte-csv/dimred_clstr.csv"
df = pd.read_csv(data_path)

In [3]:
metadata = ['Image', 'Object ID', 'Classification', 'Parent', 'Centroid X µm', 'Centroid Y µm', 'Slide', 'UMAP1_hcorr', 'UMAP2_hcorr', 'Leiden']
pcs = ['PC1_hcorr', 'PC2_hcorr', 'PC3_hcorr', 'PC4_hcorr', 'PC5_hcorr', 'PC6_hcorr', 'PC7_hcorr', 'PC8_hcorr', 'PC9_hcorr', 'PC10_hcorr']
markers = ['DAPI', 'CD44', 'HLA-DR', 'CD4', 'IFNG', 'Ki67', 'CD107a', 'CD45', 'CD20', 'CD40', 'CD8', 'Pan-Cytokeratin', 'CD68', 'HLA-A', 'CD79a', 'CD45RO', 'CD21', 'CD11c', 'HLA-E', 'IDO1', 'CD14', 'CD56', 'VISTA', 'FOXP3', 'Granzyme B', 'PCNA', 'T-bet/TBX21', 'PD-L1', 'TOX', 'PD-1', 'CD38', 'ICOS', 'CD39', 'LAG3', 'TCF-1', 'CD3e']

### Hierarchical Clustering
Based on Figure S1C of [this](https://pubmed.ncbi.nlm.nih.gov/37059105/) paper.

In [4]:
# level 1 classification
def classify_cells(row):
    """ Level 1 classification. """
    s = row['Classification']
    if ('Pan-Cytokeratin' in s) and ('CD45' not in s):
        return 'Epithelial'
    elif ('CD45' in s) and ('Pan-Cytokeratin' not in s):
        return 'Immune'
    else:
        if ('CD45' in s) and ('Pan-Cytokeratin' in s): # not exclusively one or the other
            if row['Pan-Cytokeratin'] > row['CD45']:
                return 'Epithelial'
            else:
                return 'Immune'
        else: # neither
            return 'Other'

# level 2 classification
def classify_level2(row):
    """ Level 2 classification. """
    s = row['Classification']
    lymphoid = ('CD3e' in s) or ('CD20' in s) or ('CD21' in s)
    myeloid = ('CD68' in s) or ('CD14') or ('CD11c' in s) or (('CD107a' in s) and ('CD56' in s)) or ('HLA-DR' in s) or ('HLA-A' in s) or ('VISTA' in s)
    if row['Level 1'] == 'Immune':
        if lymphoid and (not myeloid):
            return 'Lymphoid'
        elif (myeloid) and (not lymphoid):
            return 'Myeloid'
        elif lymphoid and myeloid: # not exclusively one or the other
            l = max(row['CD3e'], row['CD20'], row['CD21'])
            m = max(row['CD68'], row['CD14'], row['CD11c'], row['CD107a'], row['CD56'], row['HLA-DR'], row['HLA-A'], row['VISTA'])
            if l > m:
                return 'Lymphoid'
            else:
                return 'Myeloid'
        else:
            return 'Other'
    else:
        return 'Other'

# level 3 classification
def classify_level3(row):
    """ Level 3 classification. """
    s = row['Classification']
    cytotoxic = ('CD3e' in s) and ('CD8' in s)
    helper = ('CD3e' in s) and ('CD4' in s)
    b = ('CD20' in s) or ('CD21' in s)
    macrophage = 'CD68' in s
    monocyte = 'CD14' in s
    dendritic = 'CD11c' in s
    nk = ('CD107a' in s) and ('CD56' in s)
    apc = ('HLA-DR' in s) or ('HLA-A') in s
    chkpt = 'VISTA' in s
    if row['Level 2'] == 'Lymphoid':
        if cytotoxic and (not helper or b):
            return 'Cyt T'
        elif helper and (not cytotoxic or b):
            return 'Helper T'
        elif b and (not helper or cytotoxic):
            return 'B Cell'
        elif helper or cytotoxic or b: # not one, but not none
            max_val = row[['CD3e', 'CD8', 'CD4', 'CD20', 'CD21']].idxmax()
            if (max_val == 'CD3e'):
                # find the max between CD8 and CD4
                sub_max = row[['CD8', 'CD4']].idxmax()
                if sub_max == 'CD8':
                    return 'Cyt T'
                else:
                    return 'Helper T'
            elif (max_val == 'CD20') or (max_val == 'CD21'):
                return 'B Cell'        
        else: # none
            return 'Other'
    elif row['Level 2'] == 'Myeloid':
        if macrophage and not (monocyte or dendritic or nk or apc or chkpt):
            return 'Macrophage'
        elif dendritic and not (monocyte or macrophage or nk or apc or chkpt):
            return 'Dendritic'
        elif nk and not (macrophage or dendritic or monocyte or apc or chkpt):
            return 'NK Cell'
        elif monocyte and not (macrophage or dendritic or nk or apc or chkpt):
            return 'Monocyte'
        elif apc and not (macrophage or dendritic or monocyte or nk or chkpt):
            return 'Other Ant Pres Cell'
        elif chkpt and not (macrophage or dendritic or monocyte or apc or nk):
            return 'Imm Check+ Myeloid'
        elif macrophage or dendritic or monocyte or nk or apc or chkpt: # not one, but not none
            max_val = row[['CD68', 'CD14', 'CD11c', 'CD107a', 'CD56', 'HLA-DR', 'HLA-A', 'VISTA']].idxmax()
            if max_val == 'CD68':
                return 'Macrophage'
            elif max_val == 'CD14':
                return 'Monocyte'
            elif max_val == 'CD11c':
                return 'Dendritic'
            elif (max_val == 'CD107a') or (max_val == 'CD56'):
                return 'NK Cell' 
            elif (max_val == 'HLA-DR') or (max_val == 'HLA-A'):
                return 'Other Ant Pres Cell' 
            elif max_val == 'VISTA':
                return 'Imm Check+ Myeloid'
        else: # none 
            return 'Other'
    else:
        return 'Other'
    
# level 4 classification
def classify_level4(row):
    """ Level 4 classification. """
    s = row['Classification']
    pre = 'TCF-1' in s
    prolif = ('Ki67' in s) or ('PCNA' in s)
    act_tc = ('Granzyme B' in s) or ('CD44' in s) or ('IFNG' in s)
    tc = 'PD-1' in s
    early_ex_tc = ('TOX' in s)
    ex_tc = ('CD38' in s) or ('LAG3' in s)

    treg = 'FOXP3' in s
    act_th = ('ICOS' in s) or ('IFNG' in s)
    mem_th = 'CD45RO' in s
    th1 = 'T-bet/TBX21' in s

    plasma = 'CD38' in s
    act_b = 'HLA-E' in s
    b_ant = 'CD79a' in s

    act_mac = 'IDO1' in s
    apc = ('HLA-DR' in s) or ('HLA-A' in s)
    inhib = 'VISTA' in s
    act_den = 'CD40' in s
    act_nk = 'Granzyme B' in s
    ifg_nk = 'IFNG' in s
    
    if row['Level 3'] == 'Cyt T':
        if pre and not (prolif or act_tc or tc or early_ex_tc or ex_tc):
            return 'Cyt T Pre'
        elif prolif and not (pre or act_tc or tc or early_ex_tc or ex_tc):
            return 'Prolif Cyt T'
        elif act_tc and not (prolif or pre or tc or early_ex_tc or ex_tc):
            return 'Act Cyt T'
        elif tc and not (prolif or act_tc or pre or early_ex_tc or ex_tc):
            return 'PD-1+ Tc'
        elif early_ex_tc and not (prolif or act_tc or tc or ex_tc or pre):
            return 'Early Ex Tc'
        elif ex_tc and not (prolif or act_tc or tc or early_ex_tc or pre):
            return 'Ex Tc'
        elif pre or prolif or act_tc or tc or early_ex_tc or ex_tc: # not one, but not none
            max_val = row[['TCF-1', 'Ki67', 'PCNA', 'Granzyme B', 'CD44', 'IFNG', 'PD-1', 'TOX', 'CD38', 'LAG3']].idxmax()
            if max_val == 'TCF-1':
                return 'Cyt T Pre'
            elif (max_val == 'Ki67') or (max_val == 'PCNA'):
                return 'Prolif Cyt T'
            elif (max_val == 'Granzyme B') or (max_val == 'CD44') or (max_val == 'IFNG'):
                return 'Act Cyt T'
            elif (max_val == 'PD-1'):
                return 'PD-1+ Tc'
            elif (max_val == 'TOX'):
                return 'Early Ex Tc'
            else:
                return 'Ex Tc'
        else: # none
            return 'Other'
    elif row['Level 3'] == 'Helper T':
        if pre and not (treg or prolif or act_th or mem_th or th1):
            return 'Helper T Pre'
        elif treg and not (pre or prolif or act_th or mem_th or th1):
            return 'T reg'
        elif prolif and not (treg or pre or act_th or mem_th or th1):
            return 'Prolif Cyt T'
        elif act_th and not (treg or prolif or pre or mem_th or th1):
            return 'Act Helper T'
        elif mem_th and not (treg or prolif or act_th or pre or th1):
            return 'Mem Helper T'
        elif th1 and not (treg or prolif or act_th or mem_th or pre):
            return 'Th1 Helper T'
        elif pre or treg or prolif or act_th or mem_th or th1: # not one but not none
            max_val = row[['TCF-1', 'FOXP3', 'Ki67', 'PCNA', 'ICOS', 'IFNG', 'CD45RO', 'T-bet/TBX21']].idxmax()
            if max_val == 'TCF-1':
                return 'Helper T Pre'
            elif max_val == 'FOXP3':
                return 'T reg'
            elif (max_val == 'Ki67') or (max_val == 'PCNA'):
                return 'Prolif Cyt T'
            elif (max_val == 'ICOS') or (max_val == 'IFNG'):
                return 'Act Helper T'
            elif max_val == 'CD45RO':
                return 'Mem Helper T'
            elif max_val == 'T-bet/TBX21':
                return 'Th1 Helper T'
        else: # none
            return 'Other'
        
    elif row['Level 3'] == 'B Cell':
        if prolif and not (plasma or act_b or b_ant):
            return 'Prolif B'
        elif plasma and not (prolif or act_b or b_ant):
            return 'Plasma B'
        elif act_b and not (prolif or plasma or b_ant):
            return 'Act B'
        elif b_ant and not (prolif or plasma or act_b):
            return 'B Ant Rec'
        elif prolif or plasma or act_b or b_ant: # not one, but not none
            max_val = row[['Ki67', 'PCNA', 'CD38', 'HLA-E', 'CD79a']].idxmax()
            if (max_val == 'Ki67') or (max_val == 'PCNA'):
                return 'Prolif B'
            elif max_val == 'CD38':
                return 'Plasma B'
            elif max_val == 'HLA-E':
                return 'Act B'
            else:
                return 'B Ant Rec'
        else: # none
            return 'Other'
        
    elif row['Level 3'] == 'Macrophage':
        if act_mac and not apc:
            return 'Act Macrophage'
        elif apc and not act_mac:
            return 'Ant Pres Macrophage'
        elif apc and act_mac: # not one or the other exclusively
            max_val = row[['IDO1', 'HLA-DR', 'HLA-A']].idxmax()
            if max_val == 'IDO1':
                return 'Act Macrophage'
            else:
                return 'Ant Pres Macrophage'
        else: # neither
            return 'Other'
        
    elif row['Level 3'] == 'Monocyte':
        if inhib and not apc:
            return 'Inhib Monocyte'
        elif apc and not inhib:
            return 'Ant Pres Macrophage'
        elif apc and inhib: # not one or the other exclusively
            max_val = row[['VISTA', 'HLA-DR', 'HLA-A']].idxmax()
            if max_val == 'VISTA':
                return 'Inhib Monocyte'
            else:
                return 'Ant Pres Monocyte'
        else: # neither
            return 'Other'
        
    elif row['Level 3'] == 'Dendritic':
        if act_den and not apc:
            return 'Act Dendritic'
        elif apc and not act_den:
            return 'Ant Pres Dendritic'
        elif apc and act_den: # not one or the other exclusively
            max_val = row[['CD40', 'HLA-DR', 'HLA-A']].idxmax()
            if max_val == 'CD40':
                return 'Act Dendritic'
            else:
                return 'Ant Pres Dendritic'
        else: # neither
            return 'Other'
        
    elif row['Level 3'] == 'NK Cell':
        if act_nk and not ifg_nk:
            return 'Act NK'
        elif ifg_nk and not act_nk:
            return 'IFNG Sec NK'
        elif ifg_nk and act_nk: # not one or the other exclusively
            if row['CD40'] > row['IFNG']:
                return 'Act NK'
            else:
                return 'IFNG Sec NK'
        else: # neither
            return 'Other'
    else:
        return 'Other'

In [5]:
df['Level 1'] = df.apply(classify_cells, axis=1)
df['Level 2'] = df.apply(classify_level2, axis=1)
df['Level 3'] = df.apply(classify_level3, axis=1)
df['Level 4'] = df.apply(classify_level4, axis=1)

In [6]:
metadata.extend(['Level 1', 'Level 2', 'Level 3', 'Level 4'])
print(metadata)

['Image', 'Object ID', 'Classification', 'Parent', 'Centroid X µm', 'Centroid Y µm', 'Slide', 'UMAP1_hcorr', 'UMAP2_hcorr', 'Leiden', 'Level 1', 'Level 2', 'Level 3', 'Level 4']


In [7]:
def determine_phenotype(row):
  """ Assign the most granular phenotype possible. """
  if row['Level 4'] != 'Other':
    return row['Level 4']
  elif row['Level 3'] != 'Other':
    return row['Level 3']
  elif row['Level 2'] != 'Other':
    return row['Level 2']
  elif row['Level 1'] != 'Other':
    return row['Level 1']
  return 'Other'  # default value if all levels are 'Other'

In [8]:
df['Phenotype'] = df.apply(determine_phenotype, axis=1)
print(df['Phenotype'].unique())

['Other' 'Ant Pres Macrophage' 'NK Cell' 'Other Ant Pres Cell'
 'Act Macrophage' 'Prolif Cyt T' 'Macrophage' 'Act NK' 'Myeloid'
 'Mem Helper T' 'Helper T Pre' 'Helper T' 'IFNG Sec NK' None 'T reg'
 'B Ant Rec' 'Prolif B' 'Act Helper T' 'B Cell' 'Imm Check+ Myeloid'
 'Act Cyt T' 'Cyt T Pre' 'PD-1+ Tc' 'Cyt T' 'Dendritic' 'Ex Tc'
 'Epithelial' 'Act Dendritic' 'Th1 Helper T' 'Monocyte' 'Plasma B'
 'Inhib Monocyte' 'Ant Pres Dendritic' 'Ant Pres Monocyte' 'Act B'
 'Early Ex Tc']


In [9]:
# make the 'Timepoint' column for ease of comparison
pre_timepoints = ['Slide 11 B1', 'Slide 11 A1', 'Slide 11 A1-1', 'Slide 9 A1', 'Slide 9 B2', 'Slide 9 A1-1']
df['Timepoint'] = df['Parent'].apply(lambda x: 'DLN pre' if x in pre_timepoints else 'DLN post')
df.head()

,Image,Object ID,Classification,Parent,Centroid X µm,Centroid Y µm,DAPI,CD44,HLA-DR,CD4,...,PC10_hcorr,UMAP1_hcorr,UMAP2_hcorr,Leiden,Level 1,Level 2,Level 3,Level 4,Phenotype,Timepoint
0,Slide9_Scan1.ome.tif,3c467323-15ef-44ac-a660-3c2db2b3422c,Other/NA,Slide 9 A1,7389.8,3296.6,4.784174,0.316634,0.606009,0.460837,...,0.324316,2.010914,0.120178,11,Other,Other,Other,Other,Other,DLN pre
1,Slide9_Scan1.ome.tif,fbd8d545-c663-4aac-b5c8-7f3e3af3a95e,Other/NA,Slide 9 A1,7544.4,3300.9,4.868290,1.136582,0.527565,0.834169,...,0.010455,2.994155,-0.110734,7,Other,Other,Other,Other,Other,DLN pre
2,Slide9_Scan1.ome.tif,82adeef0-2804-4bce-9e1f-12b11bf9fe1c,CD68,Slide 9 A1,7556.3,3302.0,4.875163,1.181973,1.022271,1.737778,...,0.028281,7.935672,-2.987383,7,Other,Other,Other,Other,Other,DLN pre
3,Slide9_Scan1.ome.tif,92bea315-98b1-44fb-bece-d79a4aeb609e,Other/NA,Slide 9 A1,7384.1,3304.0,4.688905,0.567017,0.094674,0.155378,...,0.464398,1.687337,0.925616,11,Other,Other,Other,Other,Other,DLN pre
4,Slide9_Scan1.ome.tif,14e1d90e-4615-4e68-807f-fb871b565e69,Other/NA,Slide 9 A1,7522.1,3304.4,4.515042,0.780059,0.685317,1.209587,...,0.215078,4.201704,-0.894678,7,Other,Other,Other,Other,Other,DLN pre


In [10]:
df.drop(columns=['UMAP1', 'UMAP2', 'PC1_1', 'PC2_1', 'PC3_1', 'PC4_1', 'PC5_1', 'PC6_1', 'PC7_1', 'PC8_1', 'PC9_1', 'PC10_1'])

,Image,Object ID,Classification,Parent,Centroid X µm,Centroid Y µm,DAPI,CD44,HLA-DR,CD4,...,PC10_hcorr,UMAP1_hcorr,UMAP2_hcorr,Leiden,Level 1,Level 2,Level 3,Level 4,Phenotype,Timepoint
0,Slide9_Scan1.ome.tif,3c467323-15ef-44ac-a660-3c2db2b3422c,Other/NA,Slide 9 A1,7389.8,3296.6,4.784174,0.316634,0.606009,0.460837,...,0.324316,2.010914,0.120178,11,Other,Other,Other,Other,Other,DLN pre
1,Slide9_Scan1.ome.tif,fbd8d545-c663-4aac-b5c8-7f3e3af3a95e,Other/NA,Slide 9 A1,7544.4,3300.9,4.868290,1.136582,0.527565,0.834169,...,0.010455,2.994155,-0.110734,7,Other,Other,Other,Other,Other,DLN pre
2,Slide9_Scan1.ome.tif,82adeef0-2804-4bce-9e1f-12b11bf9fe1c,CD68,Slide 9 A1,7556.3,3302.0,4.875163,1.181973,1.022271,1.737778,...,0.028281,7.935672,-2.987383,7,Other,Other,Other,Other,Other,DLN pre
3,Slide9_Scan1.ome.tif,92bea315-98b1-44fb-bece-d79a4aeb609e,Other/NA,Slide 9 A1,7384.1,3304.0,4.688905,0.567017,0.094674,0.155378,...,0.464398,1.687337,0.925616,11,Other,Other,Other,Other,Other,DLN pre
4,Slide9_Scan1.ome.tif,14e1d90e-4615-4e68-807f-fb871b565e69,Other/NA,Slide 9 A1,7522.1,3304.4,4.515042,0.780059,0.685317,1.209587,...,0.215078,4.201704,-0.894678,7,Other,Other,Other,Other,Other,DLN pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4748614,Slide12_Scan1.ome.tif,674a9d46-8dcc-4307-b0ca-38b64b11b142,Other/NA,Slide 12 C1-1,11095.8,30893.2,3.282647,0.004390,0.023912,0.569283,...,0.786778,0.364773,2.567020,16,Other,Other,Other,Other,Other,DLN post
4748615,Slide12_Scan1.ome.tif,6a7750fe-7f56-4b55-8a14-48f32f7e5c24,Other/NA,Slide 12 C1-1,11128.6,30891.5,3.640503,0.000000,0.033822,0.493537,...,0.810778,0.340237,2.652914,16,Other,Other,Other,Other,Other,DLN post
4748616,Slide12_Scan1.ome.tif,1ef4fe09-93d9-46e9-89e9-a0dbe592232c,Other/NA,Slide 12 C1-1,11107.2,30893.5,4.087466,0.014692,0.055813,0.418776,...,1.080748,0.674465,1.922926,11,Other,Other,Other,Other,Other,DLN post
4748617,Slide12_Scan1.ome.tif,63caee24-5b1b-4d44-bf39-96ca87228361,Other/NA,Slide 12 C1-1,11078.8,30895.0,3.624429,0.004689,0.040566,0.825052,...,1.102018,0.834528,1.614283,11,Other,Other,Other,Other,Other,DLN post


In [11]:
sp = "gs://mayo-nguyen-transplant/nduarte-csv/data.csv"
df.to_csv(sp, index=False)